### CNN on CIFR implementing Dense net research paper:

In [3]:
from tensorflow.keras import models, layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.callbacks import ModelCheckpoint

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
os.chdir('drive/MyDrive')

In [6]:
# Hyperparameters
num_classes = 10
weight_decay = 1e-4
num_filter = 12
compression = 0.5

In [7]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [13]:
#Image augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    horizontal_flip=True,
    rotation_range=30
)
validation_datagen = ImageDataGenerator(
    rescale = 1./255,
    width_shift_range = 0.2,
    height_shift_range = 0.2
)

train_datagenerator = train_datagen.flow(
    X_train,y_train,
    batch_size=64,
    seed = 56
)

validation_datagenerator = validation_datagen.flow(
    X_test,y_test,
    batch_size=64,
    seed = 56
)

In [14]:
# Dense Block
def denseblock1(input, num_filter = 12, dropout_rate = 0):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same',kernel_initializer="he_uniform")(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition1(input, num_filter = 12, dropout_rate = 0):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same',kernel_initializer="he_uniform")(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer1(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

num_filter = 60
l = 6
tf.keras.backend.clear_session()

input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock1(First_Conv2D, num_filter)
First_Transition = transition1(First_Block, num_filter)

Second_Block = denseblock1(First_Transition, num_filter)
Second_Transition = transition1(Second_Block, num_filter)

Third_Block = denseblock1(Second_Transition, num_filter)
Third_Transition = transition1(Third_Block, num_filter)

Last_Block = denseblock1(Third_Transition,  num_filter)
output = output_layer1(Last_Block)

model1 = Model(inputs=[input], outputs=[output])
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 60)   1620        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 60)   240         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 60)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [15]:
# callbacks
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if ((logs.get('accuracy')-logs.get('val_accuracy')>0.03) and logs.get('val_accuracy')>0.90) or logs.get('val_accuracy')>0.91:
      print("\nOverfitting begins....terminating the training")
      self.model.stop_training=True

overfitting_callback = myCallback()
filepath1 = 'checkpoint_weights/weights.h5'
checkpoint = ModelCheckpoint(filepath1, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
def scheduler(epoch, lr):
  if epoch%25==0:
    return lr/3
  else:
    return lr
callback_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)


In [ ]:
# determine Loss function and Optimizer
model1.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history1 = model1.fit(train_datagenerator,
          batch_size=64,
          epochs=300,
          verbose=1, 
          validation_data=validation_datagenerator,
          callbacks = [overfitting_callback,checkpoint,callback_scheduler])

Epoch 1/300
782/782 [==============================] - 66s 82ms/step - loss: 1.6547 - accuracy: 0.3913 - val_loss: 1.7054 - val_accuracy: 0.4234

Epoch 00001: val_accuracy improved from -inf to 0.42340, saving model to checkpoint_weights/weights.h5
Epoch 2/300
782/782 [==============================] - 63s 80ms/step - loss: 1.2939 - accuracy: 0.5366 - val_loss: 1.4108 - val_accuracy: 0.5098

Epoch 00002: val_accuracy improved from 0.42340 to 0.50980, saving model to checkpoint_weights/weights.h5
Epoch 3/300
782/782 [==============================] - 63s 80ms/step - loss: 1.1061 - accuracy: 0.6068 - val_loss: 1.0961 - val_accuracy: 0.6147

Epoch 00003: val_accuracy improved from 0.50980 to 0.61470, saving model to checkpoint_weights/weights.h5
Epoch 4/300
782/782 [==============================] - 63s 80ms/step - loss: 0.9967 - accuracy: 0.6492 - val_loss: 1.0223 - val_accuracy: 0.6440

Epoch 00004: val_accuracy improved from 0.61470 to 0.64400, saving model to checkpoint_weights/weight

## As model has saturated, therefore closing the training.

## Validation accuracy has reached to 0.9